In [10]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from astropy.io import fits
import myutils
import matplotlib.pyplot as plt

In [69]:
coords_df = pd.read_csv("./data/processed/cut_crossmatched_objects.csv", index_col=0)[['ra_des','dec_des']]
BAND_ORDER = {"g": 0, "r": 1, "i": 2, "z": 3, "Y": 4}
all_imgs_array = np.zeros((len(coords_df), 28, 28, 5))
failed_object_mask = np.zeros(len(coords_df)).astype(bool)
# NOTE: remove [:2] from iteration
for i, (coadd_id, (ra, dec)) in tqdm(enumerate(coords_df[:2].iterrows()), total=len(coords_df)):
    FAILED_OBJECT_FLAG = False
    url_list = myutils.fetch_object_urls(ra, dec)
    if len(url_list) != 5:
        FAILED_OBJECT_FLAG = True
        failed_object_mask[i]=True
        break
    obj_array = np.zeros((28,28,5), dtype=np.float32)
    for url in url_list:
        if FAILED_OBJECT_FLAG:
            break
        raw_img = fits.open(url, cache=False)[0].data
        # TODO: What to do if image opening fails? In what ways does it fail?
        processed_img = myutils.crop_image(raw_img, px=28)
        if processed_img is None:
            FAILED_OBJECT_FLAG = True
            failed_object_mask[i]=True
            break
        band = myutils.band_from_url(url)
        obj_array[:, :, BAND_ORDER[band]] = processed_img
    if not FAILED_OBJECT_FLAG:
        all_imgs_array[i,:,:,:] = obj_array

success_coadd_ids = coords_df.index[~failed_object_mask]
success_imgs_array = all_imgs_array[~failed_object_mask]

  0%|          | 2/6566 [00:41<37:57:46, 20.82s/it]


In [67]:
failed_object_mask = np.random.randint(0,200,(len(coords_df)))
failed_object_mask = np.array([0 if flag else 1 for flag in failed_object_mask]).astype(bool)
all_imgs_array = np.abs(np.random.normal(size=(len(coords_df), 28, 28, 5))) * 1e4
success_imgs_array = all_imgs_array[~failed_object_mask]
success_coadd_ids = coords_df.index[~failed_object_mask]

In [68]:
print(f'{success_coadd_ids.shape} {success_imgs_array.shape}')

(6528,) (6528, 28, 28, 5)


In [281]:
coadd_ids = pd.read_csv('../data/processed/cut_crossmatched_objects.csv', index_col=0).index
failed_object_mask = np.random.randint(0,2,len(coords_df))
success_ids = np.ma.masked_array(coords_df.to_numpy(),failed_object_mask)
print(f'{sum(failed_object_mask)} {len(success_ids[success_ids.mask].data)}')

3296 3296


In [278]:
failed_object_mask

array([0, 1, 0, ..., 1, 0, 1])